# WPO 1
[Oefening 3+4](#oef4)
***


In [1]:
import numpy as np # numerical calculations in python
from scipy.optimize import fmin, fsolve
from scipy.misc import derivative
import matplotlib.pyplot as plt # package for plotting
from matplotlib import rc
rc('text', usetex=True)
from sympy import *

# allow to show plots inline
%matplotlib inline 

<a id="oef4"></a> 
## Oefening 3+4:
Beschouw de transformaties uit vraag 3:

 $
\begin{cases}
     x = R\cos(\theta)\sin(\phi)\\
     y = R\sin(\theta)\sin(\phi)\\
     z = R\cos(\phi)
\end{cases}
$

en

$ \epsilon = \frac{4x^2 + y}{z}$

Gegeven een verzameling van N metingen ($R_{i},\theta_{i},\phi_{i}$) die gaussisch verdeeld zijn rond de theoretische waarden ($R_{t},\theta_{t},\phi_{t}$) zoals hieronder gesimuleerd. Wetende dat de covariantiematrix berekend kan worden door
### $\hat{\tilde{C}}(\hat{\vec{q_{ij}}}) = \frac{1}{N-1} \sum_{k=1}^{N} (q_{ik} - \overline{q}_{i})(q_{jk} - \overline{q}_{j})$
wordt er opnieuw de covariantiematrix $\tilde{C}(x,y,z)$ en de variantie van $\epsilon$ gevraagd.


In [2]:
pi = 3.141592654
Ris = np.random.normal(30,3,1000);
Tis = np.random.normal(40.*pi/180.,4.*pi/180.,1000);
Pis = np.random.normal(50.*pi/180.,5.*pi/180.,1000);

### Eerst symbolisch

In [3]:
x,y,z,x2,y2,z2,r,t,p,eps, sigmaR, sigmat, sigmap, sigmax, sigmay, sigmaz = symbols(r'x y z x2 y2 z2 r t p eps \sigma_{R} \sigma_t \sigma_p \sigma_x \sigma_y \sigma_z')


In [4]:
x = r * cos(t) * sin(p);
y = r * sin(t) * sin(p);
z = r * cos(p);
eps = (4*(x2**2) + y2)/(z2)

In [5]:
D = Matrix([[diff(x,r), diff(y,r), diff(z,r)],[diff(x,t), diff(y,t), diff(z,t)],[diff(x,p), diff(y,p), diff(z,p)]])
Crtp = Matrix([[sigmaR**2,0,0],[0,sigmat**2,0],[0,0,sigmap**2]])

In [6]:
D

Matrix([
[   sin(p)*cos(t),   sin(p)*sin(t),    cos(p)],
[-r*sin(p)*sin(t), r*sin(p)*cos(t),         0],
[ r*cos(p)*cos(t), r*sin(t)*cos(p), -r*sin(p)]])

In [7]:
Crtp

Matrix([
[\sigma_{R}**2,           0,           0],
[            0, \sigma_t**2,           0],
[            0,           0, \sigma_p**2]])

In [8]:
Cxyz = D.T * Crtp * D

In [9]:
Cxyz

Matrix([
[            \sigma_p**2*r**2*cos(p)**2*cos(t)**2 + \sigma_t**2*r**2*sin(p)**2*sin(t)**2 + \sigma_{R}**2*sin(p)**2*cos(t)**2, \sigma_p**2*r**2*sin(t)*cos(p)**2*cos(t) - \sigma_t**2*r**2*sin(p)**2*sin(t)*cos(t) + \sigma_{R}**2*sin(p)**2*sin(t)*cos(t), -\sigma_p**2*r**2*sin(p)*cos(p)*cos(t) + \sigma_{R}**2*sin(p)*cos(p)*cos(t)],
[\sigma_p**2*r**2*sin(t)*cos(p)**2*cos(t) - \sigma_t**2*r**2*sin(p)**2*sin(t)*cos(t) + \sigma_{R}**2*sin(p)**2*sin(t)*cos(t),             \sigma_p**2*r**2*sin(t)**2*cos(p)**2 + \sigma_t**2*r**2*sin(p)**2*cos(t)**2 + \sigma_{R}**2*sin(p)**2*sin(t)**2, -\sigma_p**2*r**2*sin(p)*sin(t)*cos(p) + \sigma_{R}**2*sin(p)*sin(t)*cos(p)],
[                                                -\sigma_p**2*r**2*sin(p)*cos(p)*cos(t) + \sigma_{R}**2*sin(p)*cos(p)*cos(t),                                                 -\sigma_p**2*r**2*sin(p)*sin(t)*cos(p) + \sigma_{R}**2*sin(p)*sin(t)*cos(p),                        \sigma_p**2*r**2*sin(p)**2 + \sigma_{R}**2*cos(p)**2]])

### Nu numeriek

In [10]:
Crtp_num = [[0,0,0],[0,0,0],[0,0,0]]
Alldata = [Ris,Tis,Pis]
gemid = [np.mean(Ris),np.mean(Tis),np.mean(Pis)]

In [11]:
for i in range(len(Crtp_num)):
    for j in range(len(Crtp_num[i])):
        for k in range(len(Ris)):
            Crtp_num[i][j] += (Alldata[i][k] - gemid[i])*(Alldata[j][k] - gemid[j])
        Crtp_num[i][j] = Crtp_num[i][j]/float(len(Ris)-1)
Crtp_num

[[9.6062445156669, 0.003303356349491199, 0.0038345991529855875],
 [0.003303356349491199, 0.005213804084085975, 0.00024184194154928953],
 [0.0038345991529855875, 0.00024184194154928953, 0.00786881005513992]]

In [12]:
D_num = D.evalf(subs={r: gemid[0],t: gemid[1], p: gemid[2]})

N(D_num,3)

Matrix([
[0.587, 0.492, 0.643],
[-14.8,  17.7,     0],
[ 14.8,  12.4, -23.1]])

In [14]:
Cxyz_num = D_num.T *Matrix(Crtp_num) * D_num
N(Cxyz_num,3)

Matrix([
[ 6.09,  2.95, 0.966],
[ 2.95,  5.39, 0.706],
[0.966, 0.706,  8.05]])

### correlaticoefficient